# Loading data

In [45]:
import pandas as pd
import numpy as np
import geopy
import geopandas
import csv, json
from geojson import Feature, FeatureCollection, Point

data = pd.read_csv('usgs_phos.csv', low_memory=False)

In [ ]:
# just a toy to sample the data
# data_little = pd.read_csv('usgs_phos.csv', nrows=12)
# data_little

In [ ]:
# exploratory data analysis
# data.columns
# data.shape[0] #1758481

In [ ]:
# df['column2'] = np.where((df['column2'] == 'Null') | (df['column2'] == 0), df['column1'], df['column2'])

# Shaping Data

In [46]:
# creating a new dataframe that is condensed and only containing the fields we need, starting with 1758481 rows

data_condensed = data[['OrganizationIdentifier','ActivityMediaSubdivisionName','ActivityStartDate','ResultMeasureValue','ResultMeasure/MeasureUnitCode','ActivityLocation/LatitudeMeasure','ActivityLocation/LongitudeMeasure','HydrologicEvent']].copy()
data_condensed.shape[0]

1758481

In [47]:
# if there is an NaN in any field besides hydrologic event, drop the row. ending with 630448 rows

data_condensed.dropna(subset=['OrganizationIdentifier','ActivityMediaSubdivisionName','ActivityStartDate','ResultMeasureValue','ResultMeasure/MeasureUnitCode','ActivityLocation/LatitudeMeasure','ActivityLocation/LongitudeMeasure'],inplace=True)
data_condensed.shape[0] #630448

630448

In [48]:
# convert the ResultMeasureValue field to numeric, coerce errors so that the values that look like "<0.05" show up as 0

data_condensed['ResultMeasureValue'] = pd.to_numeric(data_condensed['ResultMeasureValue'], errors='coerce')
data_condensed['ResultMeasureValue'] = data_condensed['ResultMeasureValue'].fillna(0)
data_condensed

# Mitigating Unit Measure Issues

In [49]:
# dealing with differing units

# mg (milligrams) per liter = mg per 1,000cc and 1000 cc of water weighs 1 kg. Therefore, 1 mg/L is the same as 1 mg/kg if you are talking about water. (ResearchGate, https://www.researchgate.net/post/Converting_mg_l_to_mg_kg#:~:text=mg%20(milligrams)%20per%20liter%20%3D,you%20are%20talking%20about%20water.)

# convert ug/L to mg/L --> divide by 1000

data_condensed['ResultMeasure/MeasureUnitCode'].unique()
data_condensed['ResultMeasure/MeasureUnitCode'].value_counts()

ResultMeasure/MeasureUnitCode
mg/L     579065
ug/L      51130
ug          158
mg/kg        95
Name: count, dtype: int64

In [50]:
### --- investigation 'ug' only --- ###

# 0.325 to about 1.6, surface water for the 'ug'
# looks like this UG code is in OrganizationIdentifier called "21VASWCB", which is the VIRGINIA DEPARTMENT OF ENVIRONMENTAL QUALITY. confirmed via value_counts that this is the only org using 'ug'
ug = data_condensed.loc[data_condensed['ResultMeasure/MeasureUnitCode'] == 'ug']
ug['OrganizationIdentifier'].value_counts() #158
ug['ResultMeasureValue'].max() #1.62
ug['ResultMeasureValue'].min() #0.325

0.325

In [52]:
# how comparable are ug and ug/L? are they of the same scale? we can treat these differently
ugL = data_condensed.loc[data_condensed['ResultMeasure/MeasureUnitCode'] == 'ug/L']
# ugL['ResultMeasureValue'].max() #2971.9
# ugL['ResultMeasureValue'].min() #-0.948print(ugL.iloc[2])

In [57]:
# looking at one specific row, 2, current value is 1.234
print(ugL.iloc[2])

OrganizationIdentifier                   EPA_GLNPO
ActivityMediaSubdivisionName         Surface Water
ActivityStartDate                       2010-04-06
ResultMeasureValue                           1.234
ResultMeasure/MeasureUnitCode                 ug/L
ActivityLocation/LatitudeMeasure         44.000417
ActivityLocation/LongitudeMeasure       -82.350283
HydrologicEvent                                NaN
Name: 376678, dtype: object


In [58]:
# convert ug/L to mg/L, divide all cells by 1000 if ResultMeasureUnit = ug/L else ResultMeasureValue

data_condensed['ResultMeasureValue'] = data_condensed.apply(lambda x: x['ResultMeasureValue']/1000 if x['ResultMeasure/MeasureUnitCode']=='ug/L' else x['ResultMeasureValue'], axis=1)

In [59]:
# confirmed! value was divided by 1000

ugL = data_condensed.loc[data_condensed['ResultMeasure/MeasureUnitCode'] == 'ug/L']
print(ugL.iloc[2])

OrganizationIdentifier                   EPA_GLNPO
ActivityMediaSubdivisionName         Surface Water
ActivityStartDate                       2010-04-06
ResultMeasureValue                        0.001234
ResultMeasure/MeasureUnitCode                 ug/L
ActivityLocation/LatitudeMeasure         44.000417
ActivityLocation/LongitudeMeasure       -82.350283
HydrologicEvent                                NaN
Name: 376678, dtype: object


In [62]:
# now fixing units for the values that were changed, from ug/L to mg/L & mg/kg to mg/L

data_condensed['ResultMeasure/MeasureUnitCode'].replace('ug/L','mg/L',inplace=True)
data_condensed['ResultMeasure/MeasureUnitCode'].replace('mg/kg','mg/L',inplace=True)

C:\Users\caraca\AppData\Local\Temp\1\ipykernel_17200\1040678648.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_condensed['ResultMeasure/MeasureUnitCode'].replace('ug/L','mg/L',inplace=True)
C:\Users\caraca\AppData\Local\Temp\1\ipykernel_17200\1040678648.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [63]:
# confirming that worked

data_condensed['ResultMeasure/MeasureUnitCode'].value_counts()

ResultMeasure/MeasureUnitCode
mg/L    630290
ug         158
Name: count, dtype: int64

In [ ]:
#TODO: what do we want to do with the "ug"? drop them? think that they're a typo in VA?

# Adding in the ZIPs

In [ ]:
#creating a further condensed data set with just the lat and lon points
df2 = pd.DataFrame()
df2['Lat'] = data_condensed['ActivityLocation/LatitudeMeasure']
df2['Lon'] = data_condensed['ActivityLocation/LongitudeMeasure']

In [ ]:
# there is ratelimiting on this package according to its terms of use. about 2000 requests take 15 mintues... we have 630K rows. showing what *didn't* work

# def get_zipcode(df, geolocator, lat_field, lon_field, attempt=1, max_attempts=100):
#     try:
#         location = geolocator.reverse((df[lat_field], df[lon_field]), timeout=None)
#         return location.raw['address']['postcode']
#     except KeyError:
#         pass
#     except GeocoderTimedOut:
#         if attempt <= max_attempts:
#             return get_zipcode (df, attempt=attempt+1)
#         raise
#
# geolocator = geopy.Nominatim(user_agent='cara-umsi')
# zipcodes = df2.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Lat', lon_field='Lon')

In [ ]:
# note that in this csv, i renamed the fields to 'latitutde', 'longitude', and 'position', copying the index. later, to make the geopandas work, i had to delete the field names
df2.to_csv('latlong.csv')

In [ ]:
# reading the csv, transforming it to geojason (remember i deleted column labels, probably could've just started to read at row 1...
features = []
with open('latlong.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for latitude, longitude, position in reader:
        latitude, longitude = map(float, (latitude, longitude))
        features.append(
            Feature(
                geometry = Point((longitude, latitude)),
                properties = {
                    'position': position,
                }
            )
        )

collection = FeatureCollection(features)
with open("GeoObs.json", "w") as f:
    f.write('%s' % collection)

In [ ]:
# this chunk takes the geojson file generated above and sjoins it (finding intersections) with zipcode data
# https://stackoverflow.com/questions/48586647/python-script-to-convert-csv-to-geojson
# get zip code data from ArcGIS https://www.arcgis.com/home/item.html?id=8d2012a2016e484dafaac0451f9aea24
# first need to extract the .lpk zip codes file locally, then run the below

points = geopandas.read_file('GeoObs.json')
zipcodes = geopandas.read_file("zip_poly.gdb")
zip_points = points.sjoin(zipcodes, how='left', )

In [64]:
zip_points

,position,geometry,index_right,ZIP_CODE,PO_NAME,STATE,POPULATION,POP_SQMI,SQMI,Shape_Length,Shape_Area
0,375300,POINT (-78.68400 38.87030),6790.0,22824,Edinburg,VA,5899.0,62.20,94.84,1.260412,0.025488
1,375302,POINT (-93.36240 44.92690),18094.0,55426,Minneapolis,MN,27234.0,3885.02,7.01,0.288734,0.002071
2,375303,POINT (-78.48670 37.25750),7146.0,23958,Pamplin,VA,3142.0,30.10,104.38,1.281059,0.027463
3,375305,POINT (-93.26861 43.51392),18306.0,56036,Glenville,MN,1726.0,15.08,114.48,1.084185,0.033023
4,375308,POINT (-94.77260 46.93120),18582.0,56467,Nevis,MN,2580.0,17.10,150.90,1.706309,0.046198
...,...,...,...,...,...,...,...,...,...,...,...
630443,1758474,POINT (-77.48340 37.53200),6940.0,23220,Richmond,VA,40197.0,7471.56,5.38,0.249311,0.001420
630444,1758475,POINT (-81.14030 36.81560),7320.0,24382,Wytheville,VA,13837.0,71.73,192.91,1.933335,0.050549
630445,1758476,POINT (-81.98170 36.89940),7267.0,24266,Lebanon,VA,8506.0,74.56,114.09,1.861061,0.029861
630446,1758477,POINT (-82.49140 36.63920),7261.0,24251,Gate City,VA,8524.0,75.78,112.48,1.688213,0.029360


# Merging DFs Together

In [69]:
# first change dtype from object to int for position
zip_points['position']=zip_points['position'].astype(int)

In [70]:
# traditional inner join to merge the 2 data frames
data_usgs = pd.merge(data_condensed,zip_points,left_index=True, right_on='position')

In [71]:
# voila, chef's kiss
data_usgs

,OrganizationIdentifier,ActivityMediaSubdivisionName,ActivityStartDate,ResultMeasureValue,ResultMeasure/MeasureUnitCode,ActivityLocation/LatitudeMeasure,ActivityLocation/LongitudeMeasure,HydrologicEvent,position,geometry,index_right,ZIP_CODE,PO_NAME,STATE,POPULATION,POP_SQMI,SQMI,Shape_Length,Shape_Area
0,21VASWCB,Surface Water,2010-10-28,0.0100,mg/L,38.870300,-78.684000,NaN,375300,POINT (-78.68400 38.87030),6790.0,22824,Edinburg,VA,5899.0,62.20,94.84,1.260412,0.025488
1,MNPCA,Surface Water,2010-09-01,0.0610,mg/L,44.926900,-93.362400,NaN,375302,POINT (-93.36240 44.92690),18094.0,55426,Minneapolis,MN,27234.0,3885.02,7.01,0.288734,0.002071
2,21VASWCB,Surface Water,2010-04-28,0.0200,mg/L,37.257500,-78.486700,NaN,375303,POINT (-78.48670 37.25750),7146.0,23958,Pamplin,VA,3142.0,30.10,104.38,1.281059,0.027463
3,MNPCA,Surface Water,2010-06-24,0.4540,mg/L,43.513917,-93.268611,NaN,375305,POINT (-93.26861 43.51392),18306.0,56036,Glenville,MN,1726.0,15.08,114.48,1.084185,0.033023
4,MNPCA,Surface Water,2010-08-16,0.0140,mg/L,46.931197,-94.772603,NaN,375308,POINT (-94.77260 46.93120),18582.0,56467,Nevis,MN,2580.0,17.10,150.90,1.706309,0.046198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630443,21VASWCB,Surface Water,2024-01-03,0.0112,mg/L,37.532000,-77.483400,NaN,1758474,POINT (-77.48340 37.53200),6940.0,23220,Richmond,VA,40197.0,7471.56,5.38,0.249311,0.001420
630444,21VASWCB,Surface Water,2024-02-08,0.0200,mg/L,36.815600,-81.140300,NaN,1758475,POINT (-81.14030 36.81560),7320.0,24382,Wytheville,VA,13837.0,71.73,192.91,1.933335,0.050549
630445,21VASWCB,Surface Water,2024-02-12,0.0300,mg/L,36.899400,-81.981700,NaN,1758476,POINT (-81.98170 36.89940),7267.0,24266,Lebanon,VA,8506.0,74.56,114.09,1.861061,0.029861
630446,21VASWCB,Surface Water,2024-01-10,0.0300,mg/L,36.639200,-82.491400,NaN,1758477,POINT (-82.49140 36.63920),7261.0,24251,Gate City,VA,8524.0,75.78,112.48,1.688213,0.029360
